Test your pitch accent accuracy on a sentence
==============================================

**THIS IS AN EXPERIMENTAL PROJECT, THINGS MAY 

In [ ]:
from onsei import compare
from ipywebrtc import AudioRecorder, CameraStream, AudioStream
from ipywidgets import Dropdown
from IPython.display import Audio
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (10,10)

camera = CameraStream(constraints={'audio': True, 'video': False})
recorder = AudioRecorder(stream=camera)

teacher_filename = "data/ps/ps1_boku_no_chijin-teacher2.wav"
audio = AudioStream.from_file(teacher_filename)

In [ ]:
audio

The recording will repeat forever, click pause when you had enough :)

In [ ]:
recorder

How to record yourself:
- Click the record button on the widget above
- wait 1 sec before you start speaking than repeat the teacher's sentence.
- wait 1 sec after you are done
- Click the stop button

In [ ]:
recorder.save('test.webm')
# convert recording to wav format
!ffmpeg -hide_banner -loglevel error -y -i test.webm -ar 48000 -ac 1 test.wav
# compare with teacher, show some graphs and compute a distance to evaluate our pitch accuracy
distance = compare(teacher_filename, 'test.wav', notebook=True)

The last graph shows how different your pitch was compared to the teacher.

If some parts of the sentence are higher than others, listen again to the teacher
and focus on those part.

Once you have identified which word or mora you said with a different pitch,
you can re-record and compare yourself again.

Good luck !